<a href="https://colab.research.google.com/github/fbalicchia/SparkArrowFlight/blob/master/DuckDB_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DuckDB** introduced recently a simpler syntax for writing function with filters, for some reasons, it looks slightly like DAX syntax, this notebook is just an example how you can use it, it is not a standard SQL, it works only with DuckDB

and before you start complaining, DuckDB works fine with standard SQL, so you don't need to use the new syntax.

In [ ]:
!pip install duckdb --pre --upgrade

In [ ]:
import duckdb 
import pandas as pd
con = duckdb.connect('db')

df =con.execute('''
CREATE TABLE IF NOT EXISTS lineitem(l_returnflag string,l_linestatus  string, l_quantity numeric);
INSERT INTO lineitem VALUES ('A','Y',150), ('b','N',300), ('c','Y',200) ,('D','Y',100);
select count(*) from lineitem;

         ''').fetchdf()
print(df)
con.close()

   count_star()
0  56          


In [ ]:
import duckdb 
import pandas as pd
con = duckdb.connect('db')

try : 
    df =con.execute('''


    CREATE FUNCTION sum_all(x) as (select sum (x ) from lineitem );
    CREATE FUNCTION sum_filter(x,b,y)  as (select sum(x) filter ( b = y) from lineitem) ;
    CREATE FUNCTION Sum_default(x)  as sum(x)  ;
    CREATE FUNCTION Percentage(x)  as sum(x)/sum_all(x)  ;
         ''').fetchdf()
    print(df)
    con.close()
except:
  print('it is created already')

it is created already


In [ ]:
import duckdb 
import pandas as pd
con = duckdb.connect('db',read_only=True)


df =con.execute('''
    select * from duckdb_functions() where function_type = 'macro' and schema_name ='main' ;
               ''').fetchdf()
print(df)
con.close()

   schema_name               function_name function_type description return_type     parameters     parameter_types varargs                                                                                           macro_definition has_side_effects
0   main        list_histogram              macro         NaN         NaN         [l]            [None]              NaN     list_aggr(l, 'histogram')                                                                                  NaN            
1   main        list_mad                    macro         NaN         NaN         [l]            [None]              NaN     list_aggr(l, 'mad')                                                                                        NaN            
2   main        list_median                 macro         NaN         NaN         [l]            [None]              NaN     list_aggr(l, 'median')                                                                                     NaN            
3   main

In [ ]:
import duckdb 
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
con = duckdb.connect('db',read_only=True)
df =con.execute('''
     SET default_collation='nocase';
     select
          l_returnflag , l_linestatus
        , Sum_default(l_quantity) as sum
        , sum_filter(l_quantity,l_returnflag,'a') as flag_a
        , sum_filter(l_quantity,l_returnflag,'B') as flag_b
        , sum_filter(l_quantity,l_linestatus,'y') as status_Y
        , sum_all(l_quantity) as Qty_all
        , Percentage(l_quantity) as '%'
      from lineitem group by all ;

         ''').fetchdf()
display(df)
con.close()

,l_returnflag,l_linestatus,sum,flag_a,flag_b,status_Y,Qty_all,%
0,A,Y,1500.0,1500.0,3000.0,4500.0,7500.0,0.200000
1,b,N,3000.0,1500.0,3000.0,4500.0,7500.0,0.400000
2,c,Y,2000.0,1500.0,3000.0,4500.0,7500.0,0.266667
3,D,Y,1000.0,1500.0,3000.0,4500.0,7500.0,0.133333


At least for a simple scenario, we can have the same behavior of PowerBI DAX measures with regard to filter context, notice sum respect the filter context, the other functions just ignore it. 

You can run those functions without using any reference to tables

In [ ]:
import duckdb 
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
con = duckdb.connect('db',read_only=True)
df =con.execute('''
     SET default_collation='nocase';
     select
         sum_filter(l_quantity,l_returnflag,'a') as flag_a
        , sum_filter(l_quantity,l_returnflag,'B') as flag_b
        , sum_filter(l_quantity,l_linestatus,'y') as status_Y
        , sum_all(l_quantity) as Qty_all
              ''').fetchdf()
display(df)
con.close()

,flag_a,flag_b,status_Y,Qty_all
0,1500.0,3000.0,4500.0,7500.0
